# Hypernodes Guide

Hypernodes is a library for building robust, scalable data pipelines. It provides a structured way to define, execute, and orchestrate complex workflows, from simple functions to distributed cloud deployments.

This guide covers:
1.  **Core Concepts**: Nodes and Pipelines.
2.  **Batch Processing**: Efficient iteration with `.map()`.
3.  **State Management**: Handling heavy resources with `@stateful`.
4.  **Advanced Orchestration**: Nested pipelines and async execution.
5.  **Performance & Deployment**: `DualNode` optimization and Modal integration.

## 1. Core Concepts

### Defining Nodes
A **Node** is the fundamental unit of work in Hypernodes. It is a pure Python function decorated with `@node`.

In [1]:
from dataclasses import dataclass
from typing import List
from hypernodes import Pipeline, node


# --- Domain Objects ---
@dataclass
class Document:
    text: str
    score: float


@dataclass
class Answer:
    text: str
    sources: List[Document]


# --- Mock Services ---
class MockVectorDB:
    def search(self, query: str, k: int) -> List[Document]:
        return [Document(f"Result for {query} #{i}", 0.9 - i * 0.1) for i in range(k)]


class MockLLM:
    def generate(self, prompt: str) -> str:
        return f"LLM Answer based on: {prompt[:20]}..."


# --- Node Definitions ---


@node(output_name="retrieved_docs")
def retrieve(query: str, vector_db: MockVectorDB, top_k: int = 2) -> List[Document]:
    """Retrieve relevant documents from the vector database."""
    return vector_db.search(query, k=top_k)


@node(output_name="answer")
def generate(query: str, retrieved_docs: List[Document], llm: MockLLM) -> Answer:
    """Generate an answer using the retrieved documents."""
    context = "\n".join([d.text for d in retrieved_docs])
    response = llm.generate(f"Context: {context} Question: {query}")
    return Answer(text=response, sources=retrieved_docs)

### Building and Running Pipelines
A **Pipeline** orchestrates the execution of nodes. It automatically resolves dependencies based on input and output names.

In [2]:
# Initialize pipeline
rag_pipeline = Pipeline(nodes=[retrieve, generate])

In [3]:
rag_pipeline.visualize()

In [4]:
# Initialize resources
vector_db = MockVectorDB()
llm = MockLLM()

# Execute pipeline
result = rag_pipeline.run(
    inputs={
        "query": "What is Hypernodes?",
        "vector_db": vector_db,
        "llm": llm,
        "top_k": 2,
    }
)

print(f"Answer: {result['answer'].text}")

Answer: LLM Answer based on: Context: Result for ...


## 2. Batch Processing

Hypernodes supports efficient batch processing using the `.map()` method. This allows you to iterate over a list of inputs while keeping other inputs constant.

In [5]:
queries = ["What is Python?", "How do pipelines work?", "Is this scalable?"]

results = rag_pipeline.map(
    inputs={
        "query": queries,
        "vector_db": vector_db,  # Constant across all items
        "llm": llm,
        "top_k": 2,
    },
    map_over="query",
)

print(f"Processed {len(results)} queries.")

Processed 3 queries.


### Cartesian Product Mapping
Use `map_mode="product"` to execute the pipeline over every combination of inputs, useful for grid searches or parameter sweeps.

In [6]:
@node(output_name="score")
def evaluate_param(param_a: int, param_b: int) -> int:
    return param_a * param_b


grid_pipeline = Pipeline(nodes=[evaluate_param])

# Executes 2 * 2 = 4 combinations
grid_results = grid_pipeline.map(
    inputs={"param_a": [1, 2], "param_b": [10, 20]},
    map_over=["param_a", "param_b"],
    map_mode="product",
)
print(f"Grid results: {grid_results}")

Grid results: [{'score': 10}, {'score': 20}, {'score': 20}, {'score': 40}]


## 3. State Management & Caching

### Stateful Objects
The `@stateful` decorator allows for lazy initialization of heavy resources (like ML models). These objects are initialized once per worker and reused.

In [7]:
from hypernodes import stateful


@stateful
class HeavyModel:
    def __init__(self, model_name: str):
        print(f"Loading model '{model_name}'...")
        self.model_name = model_name

    def predict(self, x: str) -> str:
        return f"Prediction from {self.model_name} for {x}"


@node(output_name="prediction")
def predict_node(x: str, model: HeavyModel) -> str:
    return model.predict(x)


model = HeavyModel("gpt-4-turbo")
pipeline = Pipeline(nodes=[predict_node])
pipeline.run(inputs={"x": "Hello", "model": model})

Loading model 'gpt-4-turbo'...


{'prediction': 'Prediction from gpt-4-turbo for Hello'}

### Caching
Attach a cache to the pipeline to persist results. This prevents re-computation when inputs have not changed.

In [ ]:
from hypernodes import SequentialEngine
from hypernodes.cache import DiskCache

engine = SequentialEngine(cache=DiskCache(".cache"))
pipeline = Pipeline(nodes=[predict_node], engine=engine)

# Subsequent runs with the same inputs will retrieve results from cache
pipeline.run(inputs={"x": "Hello", "model": model})

{'prediction': 'Prediction from gpt-4-turbo for Hello'}

## 4. Advanced Orchestration

### Nested Pipelines
Pipelines can be encapsulated as nodes using `.as_node()`. This enables modular design and complex composition.

In [9]:
# Wrap the RAG pipeline as a single node
rag_node = rag_pipeline.as_node(
    input_mapping={"questions": "query"},  # Map outer input to inner input
    output_mapping={"answer": "generated_answer"},
    map_over="questions",  # Internal iteration
)


@node(output_name="score")
def evaluate(generated_answer: Answer) -> float:
    return len(generated_answer.text) / 100.0


eval_pipeline = Pipeline(nodes=[rag_node, evaluate])

### Async Execution
Nodes defined with `async def` are executed concurrently, making them ideal for I/O-bound operations.

In [10]:
import asyncio


@node(output_name="async_result")
async def fetch_data(url: str) -> str:
    await asyncio.sleep(0.1)
    return f"Data from {url}"


async_pipeline = Pipeline(nodes=[fetch_data])

results = async_pipeline.map(
    inputs={"url": ["google.com", "github.com", "modal.com"]}, map_over="url"
)

## 5. Performance & Deployment

### Optimization with DualNode
`DualNode` allows defining both a singular (item-by-item) and a batch implementation for a node. Engines like `DaftEngine` can leverage the batch implementation for significant performance gains.

In [11]:
from hypernodes import DualNode


def compute_singular(x: int) -> int:
    return x * 2


def compute_batch(x_series):
    # x_series is a Daft Series or Arrow Array
    return x_series * 2


fast_node = DualNode(
    output_name="doubled", singular=compute_singular, batch=compute_batch
)

try:
    from hypernodes.engines import DaftEngine
    import daft

    print("Daft installed, running DaftEngine example.")
    fast_pipeline = Pipeline(nodes=[fast_node], engine=DaftEngine())
    results = fast_pipeline.map(inputs={"x": [1, 2, 3, 4, 5]}, map_over="x")
    print(results)
except ImportError:
    print("Daft not installed, skipping DaftEngine execution example.")

Daft installed, running DaftEngine example.


🗡️ 🐟 InMemorySource: 00:00 

🗡️ 🐟 UDF compute-01a4a553-6dc0-4aaa-92b9-e832e60be4dd: 00:00 

[{'doubled': 2}, {'doubled': 4}, {'doubled': 6}, {'doubled': 8}, {'doubled': 10}]


### Observability
Use `.visualize()` to inspect the pipeline graph and `ProgressCallback` for execution tracking.

In [ ]:
try:
    from hypernodes.telemetry import ProgressCallback

    engine = SequentialEngine(callbacks=[ProgressCallback()])
    rag_pipeline_with_progress = Pipeline(nodes=[retrieve, generate], engine=engine)
    rag_pipeline_with_progress.visualize()
except ImportError:
    print("tqdm not installed, skipping ProgressCallback example.")
except Exception as e:
    print(f"Visualization skipped: {e}")

In [ ]:
rag_pipeline_with_progress.run(
    inputs={
        "query": "What is Hypernodes?",
        "vector_db": vector_db,
        "llm": llm,
        "top_k": 2,
    }
)

pipeline_4557320704   0%|          | 0/2 [00:00<?, ?it/s]

retrieve   0%|          | 0/1 [00:00<?, ?it/s]

generate   0%|          | 0/1 [00:00<?, ?it/s]

{'retrieved_docs': [Document(text='Result for What is Hypernodes? #0', score=0.9),
  Document(text='Result for What is Hypernodes? #1', score=0.8)],
 'answer': Answer(text='LLM Answer based on: Context: Result for ...', sources=[Document(text='Result for What is Hypernodes? #0', score=0.9), Document(text='Result for What is Hypernodes? #1', score=0.8)])}